In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Demo to Show GeoSeries Operations

In [1]:
import bigframes.pandas as bpd
bpd.options.display.progress_bar = None
import geopandas.array
import shapely


## Retrieve the x location (longitude)  of point geometries.

In [2]:
series = bpd.Series(
        [shapely.Point(1, 2), shapely.Point(2, 3), shapely.Point(3, 4)],
        dtype=geopandas.array.GeometryDtype()
)
series.geo.x

/usr/local/google/home/arwas/src1/python-bigquery-dataframes/bigframes/core/global_session.py:114: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return func(get_global_session(), *args, **kwargs)


0    1.0
1    2.0
2    3.0
dtype: Float64

## Retrieve the y location (latitude) of point geometries

In [3]:
series = bpd.Series(
        [shapely.Point(1, 2), shapely.Point(2, 3), shapely.Point(3, 4)],
        dtype=geopandas.array.GeometryDtype()
 )
series.geo.y

0    2.0
1    3.0
2    4.0
dtype: Float64